In [2]:
import os, sys
import pathlib
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim
!pip install pycocotools
!pip install keyboard
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

     |████████████████████████████████| 358kB 13.7MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 39 not upgraded.
Nee

In [ ]:
# CLONE DE GIT
# try:
#   if "drive/My\ Drive/models" in pathlib.Path.cwd().parts:
#     while "drive/My\ Drive/models" in pathlib.Path.cwd().parts:
#       os.chdir('/content/drive/My Drive')
#   elif not pathlib.Path('drive/My\ Drive/models').exists():
#     os.chdir('/content/drive/My Drive')
#     !git clone --depth 1 https://github.com/tensorflow/models
# except:
#   None

In [4]:
# compile proto files
# make sure you are in BASE_DIR/research/ (see cell below)
%cd /content/drive/My Drive/models/research/
# !protoc object_detection/protos/*.proto --python_out=.
# !cp object_detection/packages/tf2/setup.py .
# !python -m pip install .

/content/drive/My Drive/models/research


In [5]:
print(os.getcwd())
os.chdir("/content/drive/My Drive/models/research/object_detection")
print(os.getcwd())
try:
  DATA_DIR = os.path.join(os.getcwd(), 'data')
  MODELS_DIR = os.path.join(DATA_DIR, 'models')
  for dir in [DATA_DIR, MODELS_DIR]:
    if not os.path.exists(dir):
      os.mkdir(dir)
except:
  print('error')

/content/drive/My Drive/models/research
/content/drive/My Drive/models/research/object_detection


In [6]:
import os

In [37]:
import tarfile
import urllib.request

# os.chdir('/research')

# Download and extract model
MODEL_DATE = '20200711'
#'efficientdet_d2_coco17_tpu-32' #my primary model
MODEL_NAME = 'ssd_mobilenet_v2_320x320_coco17_tpu-8'
MODEL_TAR_FILENAME = MODEL_NAME + '.tar.gz'
MODELS_DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/'
MODEL_DOWNLOAD_LINK = MODELS_DOWNLOAD_BASE + MODEL_DATE + '/' + MODEL_TAR_FILENAME
PATH_TO_MODEL_TAR = os.path.join(MODELS_DIR, MODEL_TAR_FILENAME)
PATH_TO_CKPT = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'checkpoint/'))
PATH_TO_CFG = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'pipeline.config'))
if not os.path.exists(PATH_TO_CKPT):
  print('Downloading model. This may take a while... ', end='')
  urllib.request.urlretrieve(MODEL_DOWNLOAD_LINK, PATH_TO_MODEL_TAR)
  tar_file = tarfile.open(PATH_TO_MODEL_TAR)
  tar_file.extractall(MODELS_DIR)
  tar_file.close()
  os.remove(PATH_TO_MODEL_TAR)
  print('Done')

# Download labels file
LABEL_FILENAME = 'mscoco_label_map.pbtxt'
LABELS_DOWNLOAD_BASE = \
    'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
PATH_TO_LABELS = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, LABEL_FILENAME))
if not os.path.exists(PATH_TO_LABELS):
  print('Downloading label file... ', end='')
  urllib.request.urlretrieve(LABELS_DOWNLOAD_BASE + LABEL_FILENAME, PATH_TO_LABELS)
  print('Done')# %%bash

In [38]:
print(os.getcwd())

/content/drive/My Drive/models


In [39]:
print(PATH_TO_CFG)

/content/drive/My Drive/models/research/object_detection/data/models/ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config


In [40]:
os.chdir('/content/drive/My Drive/models/research')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
os.chdir('/content/drive/My Drive/models')
from object_detection.builders import model_builder

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

In [41]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [42]:
print(category_index)

{1: {'id': 1, 'name': 'person'}, 2: {'id': 2, 'name': 'bicycle'}, 3: {'id': 3, 'name': 'car'}, 4: {'id': 4, 'name': 'motorcycle'}, 5: {'id': 5, 'name': 'airplane'}, 6: {'id': 6, 'name': 'bus'}, 7: {'id': 7, 'name': 'train'}, 8: {'id': 8, 'name': 'truck'}, 9: {'id': 9, 'name': 'boat'}, 10: {'id': 10, 'name': 'traffic light'}, 11: {'id': 11, 'name': 'fire hydrant'}, 13: {'id': 13, 'name': 'stop sign'}, 14: {'id': 14, 'name': 'parking meter'}, 15: {'id': 15, 'name': 'bench'}, 16: {'id': 16, 'name': 'bird'}, 17: {'id': 17, 'name': 'cat'}, 18: {'id': 18, 'name': 'dog'}, 19: {'id': 19, 'name': 'horse'}, 20: {'id': 20, 'name': 'sheep'}, 21: {'id': 21, 'name': 'cow'}, 22: {'id': 22, 'name': 'elephant'}, 23: {'id': 23, 'name': 'bear'}, 24: {'id': 24, 'name': 'zebra'}, 25: {'id': 25, 'name': 'giraffe'}, 27: {'id': 27, 'name': 'backpack'}, 28: {'id': 28, 'name': 'umbrella'}, 31: {'id': 31, 'name': 'handbag'}, 32: {'id': 32, 'name': 'tie'}, 33: {'id': 33, 'name': 'suitcase'}, 34: {'id': 34, 'name'

In [43]:
import cv2
import numpy as np

In [44]:
VIDEO_NAME = 'test.mp4'
VIDEODIR = '/content/drive/My Drive/objDet/vid/' + VIDEO_NAME
OUTDIR = '/content/drive/My Drive/objDet/outvid/' + VIDEO_NAME

cap = cv2.VideoCapture(VIDEO)
res = (580 , 360)
fps = 12
out = cv2.VideoWriter(OUTDIR,cv2.VideoWriter_fourcc(*'MP4V'), fps, res)

length = 60 #in second(s)
l_fps = length * fps
i = 0
while (i < l_fps):
  # Read frame from camera
  ret, image_np = cap.read()

  if not ret:
    break

  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  #image_np_expanded = np.expand_dims(image_np, axis=0)
  
  #image_np = cv2.resize(image_np, (512, 512))

  # Things to try:
  # Flip horizontally
  # image_np = np.fliplr(image_np).copy()

  # Convert image to grayscale
  # image_np = np.tile(
  #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

  input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
  detections, predictions_dict, shapes = detect_fn(input_tensor)

  label_id_offset = 1
  image_np_with_detections = image_np.copy()

  # Indices where class == human
  #indices = np.squeeze(np.argwhere((detections['detection_classes'][0] + label_id_offset) == 1))
  indices = np.squeeze(np.argwhere((detections['detection_classes'][0].numpy() + label_id_offset).astype(int) == 1))
  boxes = detections['detection_boxes'][0].numpy()
  boxes = boxes[[indices]]  # select corresponding array indices of the 4-point box
  classes = (detections['detection_classes'][0] + label_id_offset).numpy()
  classes = classes[[indices]].astype(int)  # select corresponding array of the detected class
  scores = detections['detection_scores'][0].numpy()
  scores = scores[[indices]]  # select 

  print("\r progress: "+str(int(((i+1)/fps)))+" s", end =" ") 
  #print((detections['detection_classes'][0].numpy() + label_id_offset).astype(int))

  viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections,
    boxes,
    classes,
    scores,
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=200,
    min_score_thresh=.30,
    agnostic_mode=False)

  # Display output
  #nFrame = cv2.resize(image_np_with_detections, (800, 600))
  #cv2.imshow('object detection', nFrame)
  
  # write the frame
  out.write(image_np_with_detections)

  i = i+1

  # if cv2.waitKey(25) & 0xFF == ord('q'):
  #     break

cap.release()
out.release()
print("")
print("DONE")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a differe

 progress: 60 s 
DONE


In [15]:
###OLD
VIDEO = '/content/drive/My Drive/objDet/vid/personcount.mp4'
OUTDIR = '/content/drive/My Drive/objDet/outvid/personcount2.mp4'

cap = cv2.VideoCapture(VIDEO)
res = (580 , 360)
fps = 12
out = cv2.VideoWriter(OUTDIR,cv2.VideoWriter_fourcc(*'MP4V'), fps, res)

length = 5
l_fps = length * fps
i = 0
while (i < l_fps):
  # Read frame from camera
  ret, image_np = cap.read()

  if not ret:
    break

  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  #image_np_expanded = np.expand_dims(image_np, axis=0)
  
  #image_np = cv2.resize(image_np, (512, 512))

  # Things to try:
  # Flip horizontally
  # image_np = np.fliplr(image_np).copy()

  # Convert image to grayscale
  # image_np = np.tile(
  #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

  input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
  detections, predictions_dict, shapes = detect_fn(input_tensor)

  label_id_offset = 1
  image_np_with_detections = image_np.copy()

  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'][0].numpy(),
        (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
        detections['detection_scores'][0].numpy(),
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.60,
        agnostic_mode=False)

  # Display output
  #nFrame = cv2.resize(image_np_with_detections, (800, 600))
  #cv2.imshow('object detection', nFrame)
  
  # write the frame
  out.write(image_np_with_detections)

  try:
    print(np.squeeze(np.argwhere((detections['detection_classes'][0] + label_id_offset) == 1)))
  except:
    None

  print(i)

  i = i+1

  # if cv2.waitKey(25) & 0xFF == ord('q'):
  #     break

cap.release()
out.release()
print("DONE")

[ 1 10 11 13 14 15 17 19 20 21 23 24 26 27 28 33 34 37 44 45 48 50 52 53
 54 56 59 62 65 76 81 83 86 88 89 91 92 93 96]
0
[ 2  4  5  7  8 11 12 14 16 18 19 20 21 22 25 26 28 29 30 32 33 35 36 38
 42 46 47 50 51 53 55 59 60 62 63 65 66 67 68 71 72 73 75 77 80 81 82 83
 86 88 89 93 95 97 98]
1
[ 0  2  4  7  8  9 11 15 16 18 19 20 22 23 25 26 27 28 31 32 33 34 37 46
 49 54 55 57 58 59 60 62 64 65 68 69 70 71 73 76 77 80 83 84 87 90 91 93
 94 95 96 98]
2
[ 0  2  4  6  7  9 12 13 15 17 18 20 22 25 27 29 30 32 33 35 37 39 40 44
 45 48 54 56 58 60 62 63 64 65 66 67 70 75 77 78 79 80 84 86 90 92 95 96
 98 99]
3
[ 0  1  4  5  9 10 11 12 16 17 19 21 24 25 26 29 30 33 37 38 39 41 44 48
 49 50 51 54 55 57 59 67 68 69 70 71 73 74 76 77 78 79 80 81 84 85 87 88
 89 91 93 95 96 97]
4
[ 0  2  4  5  9 10 11 12 13 17 22 23 24 25 30 32 34 35 36 37 38 39 41 42
 44 48 49 51 52 53 54 55 56 58 59 61 63 64 65 66 67 70 75 77 78 79 81 82
 84 86 87 90 92 94 95 96]
5
[ 0  1  4  5  8 10 12 13 14 17 19 23 26 27 28 2